In [2]:
import re # expresiones regulares
import requests
from bs4 import BeautifulSoup

In [5]:
contenido_html = requests.get("https://weather.com/es-CL/tiempo/horario/l/25ef471b2bfd6c7bfba7e5df62734cb53d5ea4c92ba9c44a3e48c8e938c539b3").content
soup = BeautifulSoup(contenido_html, "lxml")

In [8]:
print(soup.find("span", class_="DetailsSummary--tempValue--1K4ka"))

<span class="DetailsSummary--tempValue--1K4ka" data-testid="TemperatureValue">24°</span>


In [20]:
# Extrayendo los dias
dias_diccionario = {}
# currentDateId0 el numero despues del "Id" va cambiando
for dia in soup.find_all("h3", attrs={"id": re.compile("^currentDateId")}):
    #print(dia)
    key=dia.get("id").split("currentDateId")[1]
    value = dia.text # obtener el solo el texto del tag por ejemplo : jueves, 7 de abril
    # creamos el diccionario
    dias_diccionario[key] = value
    


In [22]:
dias_diccionario

{'0': 'jueves, 7 de abril',
 '12': 'viernes, 8 de abril',
 '36': 'sábado, 9 de abril'}

In [23]:
# para cada dia y cada hora
datos = {
    "dia": [],
    "hora": [],
    "temperatura": [],
    "nubosidad": [],
    "precipitacion": [],
    "viento": []
}
# las ventjas de usar un diccionario es para tener la informacion ordenada 
# y luego transformarla en un data frame

In [32]:
# vamos a tomar los datos por hora en el resumen(summary no existe una id)
for datos_por_hora in soup.find_all("summary", class_="Disclosure--Summary--UuybP DaypartDetails--Summary--3IBUr Disclosure--hideBorderOnSummaryOpen--ZdSDc"):
    key = datos_por_hora.get("aria-labelledby").split()[0].split("currentDateId")[1]
    datos["dia"].append(dias_diccionario[key])
    datos["hora"].append(datos_por_hora.find("h2", attrs={"data-testid":"daypartName"}).text)
    datos["temperatura"].append(datos_por_hora.find("div", attrs={"data-testid":"detailsTemperature"}).span.text)
    datos["nubosidad"].append(datos_por_hora.find("div", attrs={"data-testid":"wxIcon"}).span.text)
    datos["precipitacion"].append(datos_por_hora.find("div", attrs={"data-testid":"Precip"}).span.text)
    datos["viento"].append(datos_por_hora.find("div", attrs={"data-testid":"wind"}).span.text)
    

In [35]:
# vamos a transformar nuestro diccionario en pandas data frame
import pandas as pd
# el diccionario en un dataframe de pandas
df = pd.DataFrame(datos)
df.head(-1)

,dia,hora,temperatura,nubosidad,precipitacion,viento
0,"jueves, 7 de abril",12:00,24°,Mayormente soleado,0%,SSO 3 km/h
1,"jueves, 7 de abril",13:00,26°,Mayormente soleado,0%,S 6 km/h
2,"jueves, 7 de abril",14:00,27°,Soleado,0%,SSO 9 km/h
3,"jueves, 7 de abril",15:00,27°,Soleado,0%,SSO 15 km/h
4,"jueves, 7 de abril",16:00,28°,Soleado,0%,SSO 15 km/h
...,...,...,...,...,...,...
138,"sábado, 9 de abril",6:00,11°,Despejado,6%,S 4 km/h
139,"sábado, 9 de abril",7:00,11°,Despejado,6%,SE 4 km/h
140,"sábado, 9 de abril",8:00,11°,Mayormente soleado,5%,S 4 km/h
141,"sábado, 9 de abril",9:00,13°,Mayormente soleado,4%,SSO 5 km/h


In [37]:
df.to_excel("datosclima.xlsx")

In [38]:
!dir

 El volumen de la unidad C es OS
 El número de serie del volumen es: 382B-EC55

 Directorio de C:\Users\MIGUEL\Desktop\PYTHON\webscraping\proyecto

07/04/2022  11:58    <DIR>          .
07/04/2022  10:28    <DIR>          ..
07/04/2022  11:08    <DIR>          .ipynb_checkpoints
07/04/2022  11:58            10.131 datosclima.xlsx
07/04/2022  11:58            10.571 datos_clima.ipynb
               2 archivos         20.702 bytes
               3 dirs  164.104.151.040 bytes libres
